# Data Wrangling

## Introduction

In this notebook, we inspect our dataset, which itself was extracted from processing the raw data packets downloaded from the source url.

After figuring out the columns of interest, we write functions that can be used to run over the entire dataset and clean the data in some standard format so that we may proceed with building our learning models.

## Raw Data Exploration

In [7]:
import pandas as pd
PATH_TO_PCAP_TSV = './extracted_pcaps/16-09-23.pcap.tsv'

In [8]:
# NOTE: For more confidence, multiple TSVs could be read
#  instead of a single pcap dump.

df = pd.read_csv(PATH_TO_PCAP_TSV, sep='\t')

/Users/admin/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,6,8,9,10,15,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
print("Total number of columns in the dataset ", df.shape[1])
print("Total number of rows in the dataset ", df.shape[0])

Total number of columns in the dataset  60
Total number of rows in the dataset  947072


## Discussion

We expect a lot of the columns in the dataset would have empty (NaN) values, or very few unique values. Since the dataset is so large (~XXGB), we choose to drop certain columns that seem to have very low data entropy. In the following cells this is what we exactly do. We figure out the columns that we would want to consider for building our learning models and vice-versa.

In [10]:
df_column_summary = df.nunique() # note that Nan counts as one additional unique value

In [11]:
low_information_columns = df_column_summary[df_column_summary <= 2].index

In [12]:
low_information_columns

Index(['tcp.flags.ack', 'tcp.flags.push', 'tcp.flags.reset', 'tcp.flags.syn',
       'tcp.flags.fin', 'tcp.checksum.status'],
      dtype='object')

In [13]:
print('Columns worth looking into are \n', df.columns.difference(low_information_columns))

Columns worth looking into are 
 Index(['eth.dst', 'eth.dst.oui', 'eth.src', 'eth.src.oui', 'eth.type',
       'frame.len', 'frame.number', 'frame.protocols', 'frame.time_delta',
       'frame.time_epoch', 'frame.time_relative', 'ip.checksum.status',
       'ip.dsfield', 'ip.dsfield.dscp', 'ip.dsfield.ecn', 'ip.dst', 'ip.flags',
       'ip.flags.df', 'ip.flags.mf', 'ip.flags.rb', 'ip.frag_offset',
       'ip.hdr_len', 'ip.id', 'ip.len', 'ip.proto', 'ip.src', 'ip.ttl',
       'ip.version', 'tcp.ack', 'tcp.analysis.ack_rtt',
       'tcp.analysis.acks_frame', 'tcp.analysis.bytes_in_flight',
       'tcp.analysis.initial_rtt', 'tcp.analysis.push_bytes_sent',
       'tcp.dstport', 'tcp.hdr_len', 'tcp.len', 'tcp.nxtseq',
       'tcp.reassembled.length', 'tcp.segment', 'tcp.segment.count', 'tcp.seq',
       'tcp.srcport', 'tcp.stream', 'tcp.time_delta', 'tcp.time_relative',
       'tcp.window_size', 'tcp.window_size_scalefactor',
       'tcp.window_size_value', 'udp.dstport', 'udp.length', 'ud

Now we proceed on to create the list of the columns we are interested in.

In [14]:
ip_cols_to_consider = ['ip.version', 
                         'ip.hdr_len', 
                         'ip.dsfield', 
                         'ip.dsfield.dscp',
                         'ip.dsfield.ecn', 
                         'ip.len', 
                         'ip.id',
                         'ip.dst', 
                         'ip.src', 
                         'ip.flags', 
                         'ip.flags.rb', 
                         'ip.flags.df', 
                         'ip.flags.mf', 
                         'ip.frag_offset', 
                         'ip.ttl', 
                         'ip.proto', 
                         'ip.checksum.status']

tcp_cols_to_consider = ['tcp.srcport', #int
                        'tcp.dstport', 
                        'tcp.stream', 
                        'tcp.seq', 
                        'tcp.nxtseq', 
                        'tcp.ack', 
                        'tcp.hdr_len', 
                        #'tcp.flags', 
                        #'tcp.flags.res', #boolean
                        #'tcp.flags.cwr', 
                        #'tcp.flags.ecn', 
                        #'tcp.flags.urg', 
                        'tcp.flags.ack', 
                        'tcp.flags.push', 
                        'tcp.flags.reset', 
                        'tcp.flags.syn', 
                        'tcp.flags.fin', 
                        #'tcp.flags.str',#str
                        'tcp.window_size_value', #int
                        'tcp.window_size', 
                        'tcp.window_size_scalefactor', 
                        'tcp.checksum.status', #int 
                        #'tcp.analysis', 
                        #'tcp.analysis.flags', 
                        'tcp.len', 
                        'tcp.analysis.acks_frame', 
                        'tcp.analysis.bytes_in_flight', 
                        'tcp.analysis.push_bytes_sent', 
                        'tcp.analysis.ack_rtt', 
                        'tcp.analysis.initial_rtt', 
                        #'tcp.urgent_pointer', 
                        'tcp.segment.count', 
                        'tcp.segment', 
                        #'tcp.segments', 
                        'tcp.reassembled.length', 
                        'tcp.time_relative', 
                        'tcp.time_delta', 
                        #'tcp.analysis.keep_alive', 
                        #'tcp.analysis.keep_alive_ack', 
                        #'tcp.options.mss', 
                        #'tcp.options.wscale'
                       ]

eth_cols_to_consider = ['eth.dst', 'eth.dst.oui', 'eth.src', 'eth.src.oui', 'eth.type']

udp_cols_to_consider = ['udp.srcport', 'udp.dstport', 'udp.length', 'udp.time_relative', 'udp.time_delta']

frame_cols_to_consider = ['frame.time_epoch', 'frame.time_delta', 'frame.time_relative', 
                          'frame.number', 'frame.len', 'frame.protocols']

cols_to_consider = eth_cols_to_consider + ip_cols_to_consider + tcp_cols_to_consider + \
                    udp_cols_to_consider + frame_cols_to_consider

In [15]:
print("Total columns to be in the resulting dataset: ", len(cols_to_consider))

Total columns to be in the resulting dataset:  60


## Modularizing Data Wrangling

In the previous section we brought down our dataset colums to a total of 60 (out of xx) inorder to read only the relevant columns into the memory. In this section we standardize the data types to the appropriate data type to optimize the memory usage.

### Address/Protocol lookup

There are columns such as `ip.src`, `ip.dst`, `frame.protocols`, etc. that are string identifiers. Inorder to reduce memory usage, we maintain an object with a key:int mapping for these values and proceed with standardizing the column datatypes.

In [16]:
class ValueMapping():
    def __init__(self):
        self.addresses = ['NaN']
    
    def value_to_index(self, value):
        if value in self.addresses:
            return self.addresses.index(value)
        else:
            self.addresses.append(value)
            return len(self.addresses)-1
        
    def index_to_value(self, index):
        assert index < len(self.addresses), "Index Error: index doesn't exist"
        return self.addresses[index]
    
valueMappings = ValueMapping()

In [17]:
try:
    valueMappings
except NameError:
    print("valueMappings is not defined!")
    
address_to_int = lambda _: valueMappings.value_to_index(_)

address_conversion = lambda col: pd.to_numeric(
                                col.fillna(value=0).apply(address_to_int),
                                downcast='unsigned')

protocol_conversion = lambda col: pd.to_numeric(
                                col.fillna(value=0).apply(address_to_int),
                                downcast='unsigned')

downcast_to_unsigned = lambda col: pd.to_numeric(
                                col.fillna(value=0), 
                                downcast='unsigned')

hexstring_to_int = lambda col: pd.to_numeric(
                                col.fillna(value=0).apply(
                                    lambda _: int(_.split(',')[0], 16) if type(_) == str else int(_)),
                                downcast='unsigned')


#numbers of the kind floats, str+','+str
dirtynums_to_int = lambda col: pd.to_numeric(
                                col.fillna(value=0).apply(
                                    lambda _: int(_.split(',')[0]) if type(_) == str else int(_)),
                                downcast='unsigned')


In [18]:
field_conversions = [{'col':'ip.src', 'op': address_conversion},
                    {'col':'ip.dst', 'op': address_conversion}, 
                    {'col': 'eth.dst', 'op': address_conversion}, 
                    {'col': 'eth.src', 'op': address_conversion},
                    {'col': 'eth.dst.oui', 'op': downcast_to_unsigned},
                    {'col': 'eth.src.oui', 'op': downcast_to_unsigned},
                    {'col': 'eth.type', 'op': hexstring_to_int},
                    {'col': 'ip.version', 'op': dirtynums_to_int},
                    {'col': 'ip.hdr_len', 'op': dirtynums_to_int},
                    {'col': 'ip.dsfield', 'op': hexstring_to_int},
                    {'col': 'ip.dsfield.dscp', 'op': dirtynums_to_int},
                    {'col': 'ip.dsfield.ecn', 'op': dirtynums_to_int},
                    {'col': 'ip.len', 'op': dirtynums_to_int},
                    {'col': 'ip.id', 'op': hexstring_to_int},
                    {'col': 'ip.flags', 'op': hexstring_to_int},
                    {'col': 'ip.flags.rb', 'op': dirtynums_to_int},
                    {'col': 'ip.flags.df', 'op': hexstring_to_int},
                    {'col': 'ip.flags.mf', 'op': hexstring_to_int},
                    {'col': 'ip.frag_offset', 'op': hexstring_to_int},
                    {'col': 'ip.ttl', 'op': dirtynums_to_int},
                    {'col': 'ip.proto', 'op': dirtynums_to_int},
                    {'col': 'ip.checksum.status', 'op': dirtynums_to_int},
                    {'col': 'tcp.srcport', 'op': dirtynums_to_int},
                    {'col': 'tcp.dstport', 'op': dirtynums_to_int},
                    {'col': 'tcp.hdr_len', 'op': dirtynums_to_int},
                    {'col': 'tcp.flags.ack', 'op': dirtynums_to_int},
                    {'col': 'tcp.flags.push', 'op': dirtynums_to_int},
                    {'col': 'tcp.flags.reset', 'op': dirtynums_to_int},
                    {'col': 'tcp.flags.syn', 'op': dirtynums_to_int},
                    {'col': 'tcp.flags.fin', 'op': dirtynums_to_int},
                    {'col': 'tcp.segment', 'op': dirtynums_to_int},
                    {'col': 'frame.protocols', 'op': protocol_conversion},]

In [19]:
""" These are columns that can be downcasted to ints """

float_cols = ['tcp.stream',
              'tcp.seq',
              'tcp.nxtseq',
              'tcp.ack',
              'tcp.window_size_value',
              'tcp.window_size',
              'tcp.window_size_scalefactor',
              'tcp.checksum.status',
              'tcp.len',
              'tcp.analysis.acks_frame',
              'tcp.analysis.bytes_in_flight',
              'tcp.analysis.push_bytes_sent',
              'tcp.analysis.ack_rtt',
              'tcp.analysis.initial_rtt',
              'tcp.segment.count',
              'tcp.reassembled.length',
              'tcp.time_relative',
              'tcp.time_delta',
              'udp.srcport',
              'udp.dstport',
              'udp.length',
              'udp.time_relative',
              'udp.time_delta',
              #'frame.time_epoch',
              'frame.time_delta',
              'frame.time_relative']

In [20]:
def standardize_data_types(df):
    """
    Update the PCAP dataset dataframes with standard data types.
    
    WARNING: This is an inplace method i.e. calling this function will 
     overwrite your original dataframe.
    """
    for conversion in field_conversions:
        df[conversion['col']] = conversion['op'](df[conversion['col']])

    for column in float_cols:
        df[column] = pd.to_numeric(df[column].fillna(value=0).astype('int'), 
                                   downcast='unsigned')


In [22]:
standardize_data_types(df)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947072 entries, 0 to 947071
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   eth.dst                       947072 non-null  uint16 
 1   eth.dst.oui                   947072 non-null  uint32 
 2   eth.src                       947072 non-null  uint16 
 3   eth.src.oui                   947072 non-null  uint32 
 4   eth.type                      947072 non-null  uint16 
 5   ip.version                    947072 non-null  uint8  
 6   ip.hdr_len                    947072 non-null  uint8  
 7   ip.dsfield                    947072 non-null  uint8  
 8   ip.dsfield.dscp               947072 non-null  uint8  
 9   ip.dsfield.ecn                947072 non-null  uint8  
 10  ip.len                        947072 non-null  uint16 
 11  ip.id                         947072 non-null  uint16 
 12  ip.dst                        947072 non-nul

In [29]:
# Finally we write the cleaned data set and read it when 
#  building our machine learning models.
df.to_feather(
    PATH_TO_PCAP_TSV.replace('extracted_pcaps', 
                             'feathers').replace('tsv', 
                                                 'feather'))